First check that you have pytorch installed, instructions are here: https://pytorch.org/get-started/locally/ , prefferably do it with anaconda if you can, I think that will lead to less problems down the road if we use other libraries.

If cuda toolkit isnt available and you have an nvidia gpu try to get that too (it might be contained within anaconda pytorch package): https://docs.nvidia.com/cuda/cuda-installation-guide-microsoft-windows/index.html

Also note that because the dataset is large I added it to the .gitignore, you should download it from here : https://www.kaggle.com/c/cassava-leaf-disease-classification/data, and extract it into the data/ folder of the project

*also make sure to run pip install tfrecord for reading tfrecord files*

In [ ]:
import pickle
import torch

from torch import FloatTensor, LongTensor
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, DataLoader
from matplotlib import pyplot as plt

if not torch.cuda.is_available():
    print("if you have an nvidia gpu get the cuda core package")
else:
    print("cuda is available")

Converting .tfrec tensorflow files to pytorch readable format

jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10

In [ ]:
import os
from os import path
from tfrecord.torch.dataset import TFRecordDataset
from tfrecord.torch.dataset import MultiTFRecordDataset

import tensorflow as tf


for example in tf.python_io.tf_record_iterator("./data/train_tfrecords/ld_train00-1338.tfrec"):
    print(tf.train.Example.FromString(example))
    break #print 1 example to see how it looks like


tfrecord_pattern = "./data/train_tfrecords/ld_train00-1338.tfrec"
index_pattern = "./data/train_tfrecords/ld_train00-1338.index"
splits = {
    "dataset1": 0.8,
    "dataset2": 0.2,
}
description = {"image": "byte", "image_name": "byte", "target": "int"}
#dataset = MultiTFRecordDataset(tfrecord_pattern, index_pattern,splits, description)

dataset = TFRecordDataset(tfrecord_pattern, index_pattern, description)

loader = torch.utils.data.DataLoader(dataset, batch_size=32)

# Trying to fix the below error, and find out whats happening here atm
data = next(iter(loader))
print(data)